In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import os

In [ ]:
path = os.path.dirname(__file__)

In [ ]:
pd.set_option('Display.max_rows',None) # print all the rows
pd.set_option('Display.max_columns',None) # print all the columns

In [ ]:
def pred_data():
    files_path = path+"/prediction_ARIMA" #path of objective folder
    files = os.listdir(files_path) # get all the files in the folder

    if ('.DS_Store' in files):
        files.remove('.DS_Store')

    index = 0

    y_pred = []
    for file in files: # iterator for traversing all the files
        if not os.path.isdir(file): # check if it is a file  
            f = open(files_path+"/"+file) # open the file
            data = pd.read_csv(f) 

            if (index == 0):
                y_pred = data
            else:
                y_pred = pd.concat([y_pred, data], axis = 1)
            index += 1
    
    return y_pred, y_pred.columns.values.tolist()

In [ ]:
def matrix_to_1d(data):
    n, p = data.shape
    return_list = data.iloc[:,0]
    for name, columns in data.iteritems():
        return_list = pd.concat([return_list, columns], axis = 0)
    return_list = return_list.values
    return return_list

In [ ]:
def differentiation(data):
    return_matrix = []
    for name, columns in data.iteritems():
        columns = columns.values
        diff = np.diff(columns)
        return_matrix.append(diff)
    return_matrix = pd.DataFrame(return_matrix)
    return_matrix = return_matrix.T
    return return_matrix

In [ ]:
def accuracy_change(data, pred):
    assert len(data) == len(pred)
    nb_prediction = 0 #number of times that prediction of an ant going to another chamber is correct
    tp_tn = 0 #total number of times an ant goes to another chamber
    for j in range (len(pred)):
        if (data[j] != 0) or (pred[j] != 0):
            nb_prediction += 1
            if (data[j] == pred[j]) :
                tp_tn += 1
    try: 
        return tp_tn/nb_prediction
    except Exception as reason:
        print(reason)

In [ ]:
def recall(data, pred):
    assert len(data) == len(pred)
    n = 0 #number of times that prediction of an ant going to another chamber is correct
    d = 0 #total number of times an ant goes to another chamber
    for j in range (len(pred)):
        if (data[j] != 0):
            n = n + 1
            if (pred[j] == data[j]) :
                d = d + 1
    try: 
        return d/n
    except Exception as reason:
        print(reason)

In [ ]:
def precision(data, pred):
    assert len(data) == len(pred)
    n = 0 #number of times that prediction of an ant going to another chamber is correct
    d = 0 #total number of times an ant goes to another chamber
    for j in range (len(pred)):
        if (pred[j] != 0):
            n = n + 1
            if (pred[j] == data[j]) :
                d = d + 1
    try: 
        return d/n
    except Exception as reason:
        print(reason)

In [ ]:
def f1_score(precision, recall):
    try:
        return (2* precision * recall/(precision+recall))
    except Exception as reason:
        print(reason)

In [ ]:
def evaluation(true_testing, pred_testing, y_true_diff, y_pred_diff, y_true, y_pred):
    y_true_diff = differentiation(true_testing)
    y_pred_diff = differentiation(pred_testing)

    y_true_diff = matrix_to_1d(y_true_diff)
    y_pred_diff = matrix_to_1d(y_pred_diff)
    y_pred_diff = y_pred_diff.astype(int)

    # Accuracy = (TP+TN)/(TP+FP+TN+FN)
    # Percentage of time when you correctly predict the chamber.
    from sklearn.metrics import accuracy_score
    print('Percentage of time when you correctly predict the chamber.')
    print(accuracy_score(y_true, y_pred))
    print('-----------')

    # Percentage of time when you correctly predict going to another chamber.
    acc_change = accuracy_change(y_true_diff, y_pred_diff)
    print('Percentage of time when you correctly predict going to another chamber.')
    print(acc_change)
    print('-----------')

    # recall=TP/(TP+FN)
    recl = recall(y_true_diff, y_pred_diff)
    print('recall:', recl)
    print('-----------')

    # precision=TP/(TP+FP)
    prec = precision(y_true_diff, y_pred_diff)
    print('precision:', prec)
    print('-----------')

    # f1 = 2*(precision*recall)/(precision+recall)
    f1 = f1_score(prec, recl)
    print('f1-score', f1)

### ARIMA

In [ ]:
pred, columns_name = pred_data()
pred.to_csv(path+'/data/pred_ARIMA.csv')
pred_training = pred[int(len(pred)*0.5):]
pred_testing = pred[:int(len(pred)*0.5)]
print(len(pred_testing))

1440


In [ ]:
y_pred = matrix_to_1d(pred_testing)

In [ ]:
data = pd.read_csv(path+'/data/time_series_chamber.csv')
data = data[columns_name]
true_training = data[int(len(data)*0.5):]
true_testing = data[:int(len(data)*0.5)]
print(len(true_testing))

1440


In [ ]:
y_true = matrix_to_1d(true_testing)

In [ ]:
evaluation(true_testing, pred_testing, y_true_diff, y_pred_diff, y_true, y_pred)

Percentage of time when you correctly predict the chamber.
0.9999852871939736
-----------
Percentage of time when you correctly predict going to another chamber.
0.9798387096774194
-----------
recall: 0.9898167006109979
-----------
precision: 0.9898167006109979
-----------
f1-score 0.9898167006109979


### VARMA loop

In [ ]:
pred = pd.read_csv(path+'/data/pred_VARMA_noloop.csv')
pred_training = pred[int(len(pred)*0.5):]
pred_testing = pred[:int(len(pred)*0.5)]
y_pred = matrix_to_1d(pred_testing)

In [ ]:
columns_name = pred.columns.values.tolist()
data = pd.read_csv(path+'/data/time_series_chamber.csv')
data = data[columns_name]
true_training = data[int(len(data)*0.5):]
true_testing = data[:int(len(data)*0.5)]
y_true = matrix_to_1d(true_testing)

In [ ]:
evaluation(true_testing, pred_testing, y_true_diff, y_pred_diff, y_true, y_pred)

Percentage of time when you correctly predict the chamber.
0.9998681434599156
-----------
Percentage of time when you correctly predict going to another chamber.
0.8504672897196262
-----------
recall: 0.9210526315789473
-----------
precision: 0.9173387096774194
-----------
f1-score 0.9191919191919191
